In [ ]:
# DNN_A with 1025 input and 2050 output
# DNN_C with 2050 input and 2050 output
# Half of the data is used to train DNN_A and the other half is given to trained DNN_A to generate data for DNN_C
# DNN_C has it's own cost function(customized)

In [1]:
#import libraries.
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.utils.data_utils import Sequence
# from keras.models import Sequential
# from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
import keras
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import sounddevice as sd
import tensorflow as tf
from mir_eval import separation 
from pystoi.stoi import stoi 
import h5py
from keras.callbacks import LearningRateScheduler

Using TensorFlow backend.


In [2]:
#import libraries.
import matplotlib.pyplot as plt
# from tabulate import tabulate
import time
import os
import librosa
from librosa.core import stft, istft
import time
import pickle
from keras import backend as K
from tensorflow.keras.models import load_model
# from keras.models import load_model

In [3]:
#read necessary files, contarins mixed and clean features.
def data_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed" +str(i) + ".p", "rb")
    ftr = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    temp = open( "TIMIT/Organized/concatenated/clean" +str(i) + ".p", "rb")
    target = pickle.load( temp ).T
    temp.close()
    return ftr, target

In [4]:
def phase_reading(i):
    temp = open( "TIMIT/Organized/concatenated/Mixed_phase" +str(i) + ".p", "rb")
    angle = pickle.load( temp  ) .T
    temp.close()
    #contarins target(clean) log power features.
    h5f = h5py.File('TIMIT/Organized/concatenated/phase.hdf5','r')
    ftr = h5f['phase']
    h5f.close()
    return angle

In [5]:
h_A = [1025,1025,1025]
h_C = [4100, 4100, 4100]

In [6]:
def reconstruct(wave,angle):
    recon1 = wave*np.cos(angle)+wave*np.sin(angle)*1j
#     recon = np.sqrt(np.power(10, wave))
#     recon1 = recon*np.cos(angle)+recon*np.sin(angle)*1j
    recon = librosa.core.istft((recon1.T), hop_length=512, win_length=2048, window='hann')
    return recon

In [7]:
I=0
def data_gen(batch_size, data_len):
    global I
    while True:             #this line is just because keras needs infinite generators
        for I in range(0,data_len-batch_size,batch_size): 
            h5f = h5py.File('TIMIT/Organized/concatenated/mixed.hdf5','r')
            ftr = h5f['mixed'][I:I+batch_size]
            h5f.close()
            h5f = h5py.File('TIMIT/Organized/concatenated/clean.hdf5','r')
            temp = h5f['clean'][:,I:I+batch_size]
            target = temp.T
            h5f.close()
            yield(ftr, target)

In [8]:
# X, y = data_reading(0)
X_dim= 1025
y_dim = 2050
estimator_A = Sequential()
estimator_A.add(Dense(h_A[0], input_dim = X_dim, kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(h_A[1], kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(h_A[2], kernel_initializer='normal', activation='relu'))
estimator_A.add(Dense(y_dim, kernel_initializer='normal'))
# Compile model
estimator_A.compile(loss='mean_squared_error', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [59]:
# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

# history = LossHistory()

In [26]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed.hdf5','r')
ftr_shape = h5f['mixed'].shape
h5f.close()
batch_size = 128
steps_per_epoch = int(ftr_shape[0]/(2*batch_size))

In [29]:
# ftr_shape[0]

939485

In [46]:
# gg = data_gen(batch_size, int(ftr_shape[0]/2))

In [48]:
# print(next(gg)[0].shape)

(128, 1025)


In [63]:

estimator_A.fit_generator(generator=data_gen(batch_size, int(ftr_shape[0]/2)),
                          steps_per_epoch=steps_per_epoch, 
                          epochs=50,
                          shuffle=True,
                          verbose=1)

Epoch 1/3
  33/7339 [..............................] - ETA: 13:33 - loss: 0.3368

KeyboardInterrupt: 

In [65]:
estimator_A.save('Models/Two_stage/trained_model.h5')

In [ ]:
estimator_A.save_weights('Models/Two_stage/weights.h5')

In [7]:
estimator_A = load_model('Models/Two_stage/trained_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [9]:
I=0
def data_gen2(batch_size, start_point, data_len):
    global I
    while True:             #this line is just because keras needs infinite generators
        for I in range(start_point,data_len-batch_size,batch_size): 
            h5f = h5py.File('TIMIT/Organized/concatenated/mixed.hdf5','r')
            ftr = h5f['mixed'][I:I+batch_size]
            h5f.close()
            h5f = h5py.File('TIMIT/Organized/concatenated/clean.hdf5','r')
            temp = h5f['clean'][:,I:I+batch_size]
            target = temp.T
            h5f.close()
            yield(ftr, target)

In [10]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed.hdf5','r')
ftr_shape = h5f['mixed'].shape
h5f.close()
batch_size = 128
steps = int((ftr_shape[0] - int(ftr_shape[0]/2))/(batch_size))

In [11]:
steps

3669

In [32]:
prediction = estimator_A.predict_generator(generator= data_gen2(batch_size, int(ftr_shape[0]/2), ftr_shape[0]),
                              steps= steps, 
                              max_queue_size=10, 
                              verbose=1)

3669/3669 [==============================] - 384s 105ms/step


In [43]:
sd.play(recon*10, 16000)

In [42]:
with h5py.File('TIMIT/Prediction/prediction.hdf5', "w") as f:
    dset = f.create_dataset('prediction', data=prediction)

In [ ]:
test_point_start = 469750
test_point_stop = 469850

In [13]:
h5f = h5py.File('TIMIT/Organized/concatenated/mixed0.hdf5','r')
test_input = h5f['mixed0'][:,test_point_start : test_point_stop]
h5f.close()

In [9]:
test.shape

(1025, 100)

In [10]:
h5f = h5py.File('TIMIT/Organized/concatenated/clean0.hdf5','r')
test_target = h5f['clean0'][test_point_start : test_point_stop]
h5f.close()

In [11]:
h5f = h5py.File('TIMIT/Organized/concatenated/phase0.hdf5','r')
angle = h5f['phase0'][:,test_point_start : test_point_stop]
h5f.close()

In [12]:
angle.shape

(1025, 100)

In [15]:
prediction = estimator_A.predict(test_input.T)

In [23]:
s1 = reconstruct(prediction[:,0:1025], angle.T)

In [24]:
sd.play(s1*10, 16000)

In [4]:
h5f = h5py.File('TIMIT/Organized/Mixed_FF_FFT/FFT_F0_F120.hdf5','r')
test = h5f['FFT_F0_F120.hdf5'][:,:]
h5f.close()

In [9]:
h5f = h5py.File('TIMIT/Organized/Mixed_FF_phase/phase_F0_F120.hdf5','r')
angle = h5f['phase_F0_F120.hdf5'][:,:]
h5f.close()